### The bias-variance trade-off

The performance of a machine learning model is often summarised in a single number: the expected error on unseen samples.

The expected error can be decomposed into three elements: bias, variance and irreducible error.

This notebook will use curve fitting to explore how model complexity, noise and sample size affect these three elements.


In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from sklearn import linear_model
import seaborn as sns

In [ ]:
def generate_sample(func, f_domain, mean, variance, sample_size):
    
    '''
    Generate a sample from a function with added gaussian noise.
    
    func:         the underlying function, which can take a scalar array as input and returns a scalar array of the same size.
    f_domain:     a tuple defining the input domain of the function e.g. (low, high).
                  Values will be sampled uniformly from the range [low, high). 
    mean:         mean of the noise distribution
    variance:     variance of the noise distribution
    sample_size:  the number of data points to be included in the sample
    
    return        tuple (x, f, y), 
                  where x is an array of function inputs,
                  f is the equivalent array of function values,
                  y are the function values with added noise
    '''
    
    x = np.random.uniform(f_domain[0], f_domain[1], sample_size)
    n = np.random.normal(mean, variance**0.5, sample_size)
    f = func(x)
    y = f + n
    
    return (x, f, y)

In [ ]:
def get_polynomial_features(x, order):
    temp = [x**(i+1) for i in range(order)]
    return np.stack(temp, 1)

def fit_polynomial(sample, order):
    x, f, y = sample
    features = get_polynomial_features(x, order)
    learner = linear_model.LinearRegression()
    model = learner.fit(features, y)
    return model

def estimate_expected_error(model, err_func, sample):
    x, y = sample
    y_pred = model.predict(x)    
    return err_func(y, y_pred).sum()/y.shape[0]

def squared_error(y, y_pred):
    return (y - y_pred)**2

def sin_with_drift(x):
    return x + np.sin(x)

In [ ]:
# Plot a sample, the line of the underlying function and the fitted polynomial curve.
# Try changing the order of the polynomial to see how it affects the goodness of fit.

%matplotlib inline

func = sin_with_drift
f_domain = (0, 2*np.pi)
order = 3
sample_size = 50
noise_mean = 0
noise_variance = 0.1

s = generate_sample(func, f_domain, noise_mean, noise_variance, sample_size)

x = np.linspace(f_domain[0], f_domain[1], 100, True)
features = get_polynomial_features(x, order)

m = fit_polynomial(s, order)

y = m.predict(features)
f = func(x)
plt.plot(x,f)
plt.plot(x,y, 'r')
plt.scatter(s[0], s[2])

In [ ]:
# For a given value of x, show the distribution of y values predicted by models generated from different datasets
# Plot the distribution of predictions along with their mean and the true expected y value

%matplotlib inline

func = sin_with_drift
f_domain = (0, 2*np.pi)
order = 10
sample_size = 50
noise_mean = 0
noise_variance = 0.1
x_test = 2
y_true = func(x_test)
test_features = get_polynomial_features(np.ones((1,))*x_test, order)
num_datasets = 1000

result = []
for d in range(num_datasets):
    s = generate_sample(func, f_domain, noise_mean, noise_variance, sample_size)
    m = fit_polynomial(s, order)
    y_test = m.predict(test_features)
    result.append(y_test[0])

mean_prediction = np.mean(result)
var_prediction = np.var(result)
print(var_prediction)
plt.hist(result, bins=25, normed=True, histtype='step')
plt.axvline(mean_prediction, color='r', linewidth=2)
plt.axvline(y_true, color='g', linewidth=2)

In [ ]:
sns.kdeplot(np.array(result))
plt.axvline(mean_prediction, color='r', linewidth=2)
plt.axvline(y_true, color='g', linewidth=2)

In [ ]:
%matplotlib inline

func = sin_with_drift
f_domain = (0, 2*np.pi)
order = 1
sample_size = 50
noise_mean = 0
noise_variance = 0.1
num_x = 100
x = np.linspace(f_domain[0], f_domain[1], num_x, True)
features = get_polynomial_features(x, order)
y_true = func(x)
num_datasets = 3

result = np.zeros([num_datasets, num_x])
for d in range(num_datasets):
    s = generate_sample(func, f_domain, noise_mean, noise_variance, sample_size)
    m = fit_polynomial(s, order)
    y_pred = m.predict(features)
    result[d,:] = y_pred


In [ ]:
y_true

In [ ]:
bias = np.mean((np.mean(result, 0) - y_true)**2)
variance = np.mean(np.var(result,0))

In [ ]:
variance

In [ ]:
func = np.cos
f_domain = (0, 2*np.pi)
order = 3
train_errors = []
test_errors = []

for i in range(100):
    train = generate_sample(func, f_domain, 0, 0.1, 50)
    train_f = get_polynomial_features(train[0], order)
    test = generate_sample(func, f_domain, 0, 0.1, 1000)
    test_f = get_polynomial_features(test[0], order)
    m = fit_polynomial(train, order)
    train_error = estimate_expected_error(m, squared_error, (train_f, train[2]))
    test_error = estimate_expected_error(m, squared_error, (test_f, test[2]))
    train_errors.append(train_error)
    test_errors.append(test_error)
plt.scatter(train_errors, test_errors)

In [ ]:
# Generate N training samples

# Generate a large test sample

# Fit a model to each training sample


# 